In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import re

In [ ]:
os.chdir('/content/drive/MyDrive/data/')
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
train['text'] = train['text'].apply(lambda x: word_tokenize(x))
train['words_count'] = train['text'].apply(lambda x: len(x))
train['text'] = train['text'].apply(lambda x: ' '.join([s for s in x if s not in stop_words]))
train['stop_words_count'] = train['text'].apply(lambda x: len(x))
train['numerics'] = train['text'].apply(lambda x : len([s for s in x if s.isdigit()]))

In [ ]:
train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['author','index'],1), train['author'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train['text'])
test_vectors = vectorizer.transform(X_test['text'])
print(train_vectors.shape, test_vectors.shape)

In [ ]:
%%time
import lightgbm as lgb

dtrain = lgb.Dataset(train_vectors,y_train)
dtest = lgb.Dataset(test_vectors,y_test)

param_lgb = {
    'max_depth': 5,
    'learning_rate': 0.33,
    'max_bin': 1000,
    'num_leaves' : 100,
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'softmax',
    'num_iter': 1000
    
}

lgb_model = lgb.train(param_lgb,dtrain,valid_sets=dtest)

In [ ]:
y_pred = lgb_model.predict(test_vectors)
y_argmax = y_pred.argmax(1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy=accuracy_score(y_test, y_argmax)
precision = precision_score(y_test, y_argmax, average='micro')
recall = recall_score(y_test, y_argmax, average='micro')
f1 = f1_score(y_test, y_argmax, average='micro')
print('accuracy:',accuracy,'\n','precision:',precision,'\n','recall:',recall,'\n','f1_score:',f1)